## NOTES:
[GPT-2 Classification Getting Started](https://www.kaggle.com/code/andres6garzon/getting-started-nlp-classification-using-gpt-2)

[GPT-2 (Medium)](https://huggingface.co/openai-community/gpt2-medium)

In [15]:
import pandas as pd
import time
from datasets import Dataset, load_dataset
import numpy as np
import torch
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, classification_report
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

In [26]:
langs = ['java', 'python', 'pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = load_dataset('NLBSE/nlbse25-code-comment-classification')

In [30]:
ds['java_train'][0]

{'index': 0,
 'class': 'Abfss.java',
 'comment_sentence': 'azure blob file system implementation of abstractfilesystem.',
 'partition': 0,
 'combo': 'azure blob file system implementation of abstractfilesystem. | Abfss.java',
 'labels': [1, 0, 0, 0, 0, 0, 0]}

In [3]:
def evaluate_model(model):
    total_flops = 0
    total_time = 0
    scores = []
    for lan in langs:
        with torch.profiler.profile(with_flops=True) as p:
            begin = time.time()
            for i in range(10):
                y_pred = model(ds[f'{lan}_test']['combo']).numpy().T
            total = time.time() - begin
            total_time = total_time + total
        total_flops = total_flops + (sum(k.flops for k in p.key_averages()) / 1e9)
        y_true = np.array(ds[f'{lan}_test']['labels']).T
        for i in range(len(y_pred)):
            assert(len(y_pred[i]) == len(y_true[i]))
            tp = sum([true == pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
            tn = sum([true == pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
            fp = sum([true == 0 and pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
            fn = sum([true == 1 and pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1 = (2*tp) / (2*tp + fp + fn)
            scores.append({'lan': lan, 'cat': labels[lan][i],'precision': precision,'recall': recall,'f1': f1})
    print("Compute in GFLOPs:", total_flops/10)
    print("Avg runtime in seconds:", total_time/10)
    scores = pd.DataFrame(scores)
    return scores

## Getting Started

In [12]:
# Params
seed = 42
batch_size = 8
max_length = 512
num_labels = 7
num_epochs = 1

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [22]:
class DatasetCreator(Dataset):
    def __init__(self, ds, train):
        self.ds = ds
        self.train = train
    
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        if self.train:
            return {'text': self.ds[idx]['combo'], 'label': self.ds[idx]['labels']}
        else:
            return {'text': self.ds[idx]['combo'], 'label': 0}

class GPT2_collator(object):
    def __init__(self, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, sequences):
        inputs = self.tokenizer(sequences, padding="max_length", truncation=True, max_length=self.max_length, return_tensors='pt')
        return inputs

def train(dataloader, optimizer, scheduler, device):
    global model
    model.train()
    predictions_labels = []
    true_labels = []
    total_loss = 0
    
    for batch in tqdm(dataloader, total=len(dataloader)):
        true_labels += batch['label'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device) for k,v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss, logits = outputs[:2]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        predictions_labels += logits.argmax(axis=-1).flatten().tolist()
    avg_epoch_loss = total_loss / len(dataloader)
    return predictions_labels, true_labels, avg_epoch_loss

def validate(dataloader, device):
    global model
    model.eval()
    predictions_labels = []
    true_labels = []
    total_loss = 0
    
    for batch in tqdm(dataloader, total=len(dataloader)):
        true_labels += batch['label'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device) for k,v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            loss, logits = outputs[:2]
            total_loss += loss.item()
            predictions_labels += logits.argmax(axis=-1).flatten().tolist()
    avg_epoch_loss = total_loss / len(dataloader)
    return predictions_labels, true_labels, avg_epoch_loss

def predict(dataloader, device):
    global model
    model.eval()
    predictions_labels = []
    
    for batch in tqdm(dataloader, total=len(dataloader)):
        batch = {k:v.type(torch.long).to(device) for k,v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs[0]
            predictions_labels += logits.argmax(axis=-1).flatten().tolist()
    return predictions_labels

In [ ]:
print('Loading model...')
model_config = GPT2Config.from_pretrained('gpt2', num_labels=num_labels)

print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=model_config)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.eos_token_id
model.to(device)

Loading model...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Loading tokenizer...


c:\Users\Erik\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Erik\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loading model...


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=7, bias=False)
)

In [10]:
gpt2_collator = GPT2_collator(tokenizer, 1024)

train_dataset = DatasetCreator(ds['java_train'], True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=gpt2_collator)

eval_dataset = DatasetCreator(ds['java_test'], False)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=1, shuffle=False, collate_fn=gpt2_collator)

In [23]:
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

loss = []
accuracy = []
eval_loss_list = []
eval_accuracy_list = []

for epoch in tqdm(range(num_epochs)):
    train_labels, true_labels, train_loss = train(train_dataloader, optimizer, scheduler, device)
    train_acc = accuracy_score(true_labels, train_labels)
    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss}, Train Accuracy: {train_acc}')
    loss.append(train_loss)
    accuracy.append(train_acc)
    
    eval_labels, true_labels, eval_loss = validate(eval_dataloader, device)
    eval_acc = accuracy_score(true_labels, eval_labels)
    print(f'Epoch {epoch+1}/{num_epochs} - Eval Loss: {eval_loss}, Eval Accuracy: {eval_acc}')
    eval_loss_list.append(eval_loss)
    eval_accuracy_list.append(eval_acc)
    

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/7614 [00:00<?, ?it/s]

KeyError: 'label'